# 4. Review generated data similarity(Regression)

## 라이브러리 불러오기

In [1]:
import pandas as pd
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 

In [2]:
import copy
import time

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Jupiter Cell Full Screen View
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# useful for debuging (print the results of both formulas and functions entered in one cell of Jupiter)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
# Each column width at maximum (print all column contents)
pd.set_option('display.max_colwidth', -1)
# Show up to 500 rows
pd.set_option('display.max_rows', 500)
# Display up to 500 columns
pd.set_option('display.max_columns', 500)
# Total length of data frame
pd.set_option('display.width', 1000)

print('ready to run')
# logging starttime 
startTime = time.time()

ready to run


## 함수선언하기

여러 알고리즘을 통해 모델을 학습하는 일은 반복작업이 많기 때문에 함수를 선언하여 활용합니다. 

이번 실습에서는 2개의 데이터를 각각 3가지의 모델로 학습하여 더 잦은 반복이 필요하기 때문에 함수를 통해 코드의 가독성을 높였습니다. 

### 변수 선언하기
- 전처리에 활용되는 변수를 선언합니다. 
- 카테고리 타입과 수치형 타입의 칼럼을 분리합니다. 

In [4]:
category_cols = ['dec_num','dec_custom_code','imp_dec_code','imp_trd_code','imp_typ_code',\
                 'collect_code','typ_transport_code','dec_mark','importer','ovs_cust_code',\
                 'exps_carr_code','HS10','country_ship_code','country_orig_code','trff_class_code',\
                 'country_orig_mark_code','crime_yn','key_exposure']

In [5]:
number_cols = ['trff_rate','dec_weight','taxabal_price_KRW']

### 최소최대 정규화 함수

In [ ]:
#최소 최대 정규화 식을 포함한 함수
def normalize(column):
    return (column - column.min())/(column.max() - column.min())

### 데이터 전처리를 위한 함수

In [7]:
def df_preprocessing(df):
    #데이터를 안정적으로 활용하기 위해 복사하여 활용
    copy_df = copy.deepcopy(df)

    #데이터 타입별로 데이터를 분리하여 작업하기 위한 준비
    copy_df_category = copy_df[category_cols]
    copy_df_number = copy_df[number_cols]
    
    #인코딩 객체를 불러옵니다. 
    encoder = LabelEncoder()

    #카테고리 타입의 데이터를 가진 데이터프레임을 인코딩합니다.
    for column_name,item in copy_df_category.iteritems(): 
        encoder.fit(item)
        labels = encoder.transform(item)
        copy_df_category[column_name] = labels
        
    #수치형 타입의 데이터를 가진 데이터프레임을 최소-최대 정규화합니다.
    copy_df_number_norm = copy_df_number.apply(normalize)
    
    #두가지 데이터를 각각 출력합니다.
    return copy_df_category, copy_df_number_norm

### 데이터를 훈련데이터와 테스트 데이터로 분리하는 함수

- 사이킷런에 포함된 전처리 함수인 train_test_split 를 활용합니다. 
- 타겟항목은 'crime_yn'으로 범죄 유무를 뜻합니다. 
- 학습항목 중 'key_exposure'은 타깃 'crime_yn'과 너무 높은 상관관계가 있어 다른 칼럼들이 활용되지 못하게 합니다. 
- 따라서 학습항목에서 타겟항목 'crime_yn'과 'key_expousre'은 배제합니다. 
- 마지막으로 훈련용 데이터와 테스트 데이터의 사이즈를 각각 8:2로 분리합니다. 

In [12]:
# 훈련데이터셋과 테스트데이터셋을 분리합니다. 
def df_splist(df):

    #타겟을 설정합니다.
    y = df['crime_yn']
    #타겟 데이터와 지나치게 관련성이 높은 데이터를 삭제합니다. 
    X = df.drop(columns=['crime_yn','key_exposure'])
    #데이터를 분리합니다. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

### 원본 데이터에서 상위 5% 회귀값을 가진 그룹에서의 우범률을 구하는 함수

In [8]:
def reg_top_5per(df):
    
    #상위 5%에 해당하는 데이터값을 조건식을 통해 추려내서 result 변수에 삽입
    top_5percent = df['pred'].quantile(0.95)
    result = df[df['pred'] > top_5percent]
    #타겟값에서 상위 5%에 해당하는 인덱스를 가진 값만 뽑아서 y_test_top5 변수에 삽입
    y_test_top5 = y_test[y_test.index.isin(result['index'])]
    #빠른 연산을 위해 'crime_yn'칼럼으로 y_test_top 값을 가진 데이터프레임 생성
    y_test_top5_df = pd.DataFrame({'crime_yn' :y_test_top5})
    # crime_yn ==1 인 변수의 갯수/전체 변수의 갯수를 구하여 우범률을 계산함
    base_5 = y_test_top5_df[y_test_top5_df['crime_yn']==1].count() / y_test_top5_df.count()
    
    return base_5

### 원본 데이터에서 상위 10% 회귀값을 가진 그룹에서의 우범률을 구하는 함수

In [9]:
def reg_syn_top_5per(df):
    
    #상위 5%에 해당하는 데이터값을 조건식을 통해 추려내서 result 변수에 삽입
    top_5percent = df['pred'].quantile(0.95)
    result = df[df['pred'] > top_5percent]
    #타겟값에서 상위 5%에 해당하는 인덱스를 가진 값만 뽑아서 y_test_top5 변수에 삽입
    y_syn_test_top5 = y_syn_test[y_syn_test.index.isin(result['index'])]
    #빠른 연산을 위해 'crime_yn'칼럼으로 y_test_top 값을 가진 데이터프레임 생성
    y_syn_test_top5_df = pd.DataFrame({'crime_yn' :y_syn_test_top5})
    # crime_yn ==1 인 변수의 갯수/전체 변수의 갯수를 구하여 우범률을 계산함
    base_5 = y_syn_test_top5_df[y_syn_test_top5_df['crime_yn']==1].count() / y_syn_test_top5_df.count()
    
    return base_5

### 합성 데이터에서 상위 5% 회귀값을 가진 그룹에서의 우범률을 구하는 함수

In [10]:
def reg_top_10per(df):
    
    #상위 10%에 해당하는 데이터값을 조건식을 통해 추려내서 result 변수에 삽입
    top_10percent = df['pred'].quantile(0.90)
    result = df[df['pred'] > top_10percent]
    #타겟값에서 상위 10%에 해당하는 인덱스를 가진 값만 뽑아서 y_test_top10 변수에 삽입
    y_test_top10 = y_test[y_test.index.isin(result['index'])]
    #빠른 연산을 위해 'crime_yn'칼럼으로 y_test_top 값을 가진 데이터프레임 생성
    y_test_top10_df = pd.DataFrame({'crime_yn' :y_test_top10})
    # crime_yn ==1 인 변수의 갯수/전체 변수의 갯수를 구하여 우범률을 계산함
    base_10 = y_test_top10_df[y_test_top10_df['crime_yn']==1].count() / y_test_top10_df.count()
    
    return base_10

### 합성 데이터에서 상위 10% 회귀값을 가진 그룹에서의 우범률을 구하는 함수

In [11]:
def reg_syn_top_10per(df):
    
    #상위 10%에 해당하는 데이터값을 조건식을 통해 추려내서 result 변수에 삽입
    top_10percent = df['pred'].quantile(0.90)
    result = df[df['pred'] > top_10percent]
    #타겟값에서 상위 10%에 해당하는 인덱스를 가진 값만 뽑아서 y_test_top10 변수에 삽입
    y_syn_test_top10 = y_syn_test[y_syn_test.index.isin(result['index'])]
    #빠른 연산을 위해 'crime_yn'칼럼으로 y_test_top 값을 가진 데이터프레임 생성
    y_syn_test_top10_df = pd.DataFrame({'crime_yn' :y_syn_test_top10})
    # crime_yn ==1 인 변수의 갯수/전체 변수의 갯수를 구하여 우범률을 계산함
    base_10 = y_syn_test_top10_df[y_syn_test_top10_df['crime_yn']==1].count() / y_syn_test_top10_df.count()
    
    return base_10

## 데이터 불러오기 / 데이터 전처리하기

인공지능 모델을 훈련하기 위해서는 일반적으로 훈련세트와 테스트세트로 데이터를 나눕니다.(8:2 비율로)

그리고 하나의 세트는 학습용 데이터와 타겟 데이터로 구성되어 있습니다. 
따라서 우리가 하나의 모델을 훈련시키기 위해서는 4가지의 데이터프레임이 필요합니다. 

또한, 
이번 실습에서는 학습 시킬 데이터가 원본데이터와 합성데이터로 2가지 이기 때문에 총 8개의 데이터 프레임이 생성될 것입니다. 
- 원본데이터 (X_train, X_test, y_train, y_test) -> 4가지 데이터 프레임 
- 합성데이터 (X_syn_train, X_syn_test, y_syn_train, y_syn_test) -> 4가지 데이터 프레임 

### 원본 데이터

- 원본 데이터를 불러와서 전처리 및 훈련세트와 테스트세트로 분리합니다.

In [13]:
df_base = pd.read_csv('df_syn_en.csv', encoding='utf-8-sig') #새로운 가상데이터

불러온 데이터를 복사하여 작업을 진행합니다.

In [14]:
copy_base = copy.deepcopy(df_base)

- 전처리 함수를 활용해 혼재된 데이터들을 분리 및 전처리를 실행합니다.

In [15]:
copy_base_category, copy_base_number_norm = df_preprocessing(copy_base)

#### 데이터 결합

- 전처리가 종료된 데이터를 다시 하나의 테이블로 결합합니다.

In [16]:
copy_base_total = pd.concat([copy_base_category,copy_base_number_norm], axis=1)

#### 훈련데이터, 테스트 데이터 분리

In [17]:
X_train, X_test, y_train, y_test = df_splist(copy_base_total)

### 합성 데이터

- 앞서 원본 데이터에서 진행한 데이터 전처리 및 훈련세트와 테스트세트로 분리하는 작업을 반복합니다.

In [18]:
df_syn = pd.read_csv('./data_sample/df_syn_en_17.csv', encoding='utf-8-sig') #새로운 가상데이터

불러온 데이터를 복사하여 작업을 진행합니다.

In [19]:
copy_syn = copy.deepcopy(df_syn)

In [20]:
copy_syn_category, copy_syn_number_norm = df_preprocessing(copy_syn)

#### 데이터 결합

- 전처리가 종료된 데이터를 다시 하나의 테이블로 결합합니다.

In [21]:
copy_syn_total = pd.concat([copy_syn_category,copy_syn_number_norm], axis=1)

#### 훈련데이터, 테스트 데이터 분리

In [22]:
X_syn_train, X_syn_test, y_syn_train, y_syn_test = df_splist(copy_syn_total)

## 분석하기

- 모델 평가하기

모델은 각 자료에 대한 우범 가능성을 0~1 사이의 값으로 도출하며 

상위 5%, 10%의 물품에 대한 우범률(적발률)을 평가지표로 활용하였습니다.

해당 평가지표는 원본데이터와 얼마나 유사한 우범률을 갖고 있는지와 

상위 5% 집단의 우범률과 10% 집단의 우범률의 차이를 비교하여 두 데이터셋이 얼마나 유사한 통계적 특징들이 있는지 알아보기 위해 활용되었습니다.

### 선형 회귀분석

- 선형 회귀는 종속 변수와 하나 이상의 독립 변수 간의 관계를 모델링하는 데 사용되는 통계적 방법입니다. 
- 변수 간의 선형 관계를 가정하고 최소 제곱 방법을 사용하여 회귀 매개변수를 추정합니다. 
- 선형 회귀는 다양한 분야에서 예측 및 추론에 널리 사용되지만 가정과 한계를 신중하게 고려해야 하는 특징이 있습니다. 

In [23]:
from sklearn.linear_model import LinearRegression

#### 원본데이터 (X_train, X_test, y_train, y_test)

- 원본데이터 회귀모델 학습 

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

 R<sup>2</sup>
- 회귀의  R<sup>2</sup> 스코어를 구하는 방법 입니다. 
- 0에 가까울 수록 x,y 간에 선형관계가 존재하지 않는 것을 의미합니다. 
- 해당 데이터는 독립변수와 종속변수의 선형 관계가 미약함을 확인할 수 있습니다.

In [25]:
score = lr.score(X_test, y_test)
print("Model score:", score)

Model score: 0.009805086199265345


- 원본데이터 예측값

합성데이터를 통해 학습한 모델에 테스트 데이터를 입력하여 예상되는 회귀값을 출력합니다. 

모델은 각 자료에 대한 우범 가능성을 0~1 사이의 값으로 도출하며 

상위 5%, 10%의 물품에 대한 우범률(적발률)을 평가지표로 활용하였습니다.

In [26]:
y_pred = lr.predict(X_test)
index = X_test.index.tolist()
df = pd.DataFrame({'index' : index ,'pred': y_pred})

원본데이터 우범 회귀 예측값 상위 5%의 우범률
- 예측값 중 상위 5%에 해당하는 데이터만 뽑아 우범여부를 판단합니다. 

In [27]:
base_5 = reg_top_5per(df)

 원본데이터 우범 회귀 예측값 상위 10%의 우범률
- 예측값 중 상위 10%에 해당하는 데이터만 뽑아 우범여부를 판단합니다. 

In [28]:
base_10 = reg_top_10per(df)

#### 합성데이터 (X_syn_train, X_syn_test, y_syn_train, y_syn_test)

- 합성데이터 회귀모델 학습 

In [29]:
lr_syn = LinearRegression()
lr_syn.fit(X_syn_train, y_syn_train)

LinearRegression()

In [30]:
score = lr_syn.score(X_syn_test, y_syn_test)
print("Model score:", score)

Model score: -0.02607092717158621


- 합성데이터 예측값

합성데이터를 통해 학습한 모델에 테스트 데이터를 입력하여 예상되는 회귀값을 출력합니다. 

모델은 각 자료에 대한 우범 가능성을 0~1 사이의 값으로 도출하며 

상위 5%, 10%의 물품에 대한 우범률(적발률)을 평가지표로 활용하였습니다.

In [31]:
y_syn_pred = lr_syn.predict(X_syn_test)
index = X_syn_test.index.tolist()
df = pd.DataFrame({'index' : index ,'pred': y_syn_pred})

- 합성데이터 우범 회귀 예측값 상위 5%의 우범률

In [32]:
syn_5 = reg_syn_top_5per(df)

- 합성데이터 우범 회귀 예측값 상위 10%의 우범률

In [33]:
syn_10 = reg_syn_top_10per(df)

#### 우범률 정리 - 선형 회귀분석

In [34]:
df_result_lr = pd.DataFrame({'category': 'LinearRegression','base_5' : base_5, 'base_10': base_10,'syn_5': syn_5,'syn_10': syn_10})

약간의 차이를 보이지만 5% 집단에서 10% 집단이 될때의 우범률 감소는 비슷합니다. 

이는 유사한 분포를 보이고 있다고 추론할 수 있습니다. 

In [35]:
df_result_lr

,category,base_5,base_10,syn_5,syn_10
crime_yn,LinearRegression,0.35,0.308333,0.266667,0.233333


### 랜덤포레스트

랜덤 포레스트는 정확도를 높이고 과적합을 줄이기 위해 여러 의사 결정 트리를 구축하고 예측을 집계하는 기계 학습 알고리즘입니다.


주요 특징은 다음과 같습니다.


- 각 트리를 구축하기 위한 기능 및 샘플의 무작위 선택
- 예측력 향상을 위한 앙상블 학습
- 높은 차원의 대규모 데이터 세트를 처리하는 능력.

In [36]:
from sklearn.ensemble import RandomForestRegressor

#### 원본데이터 (X_train, X_test, y_train, y_test)

- 원본데이터 랜덤포레스트 회귀모델 학습 

In [37]:
rf = RandomForestRegressor(n_estimators=50, random_state=42)

In [38]:
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=42)

- 원본데이터 예측값

In [39]:
y_pred = rf.predict(X_test)

In [40]:
index = X_test.index.tolist()

In [41]:
df = pd.DataFrame({'index' : index ,'pred': y_pred})

- 원본데이터 우범 회귀 예측값 상위 5%의 우범률

In [42]:
base_5 = reg_top_5per(df)

- 원본데이터 우범 회귀 예측값 상위 10%의 우범률

In [43]:
base_10 = reg_top_10per(df)

#### 합성데이터 (X_syn_train, X_syn_test, y_syn_train, y_syn_test)

- 합성데이터 랜덤포레스트 회귀모델 학습 

In [44]:
rf_syn = RandomForestRegressor(n_estimators=50, random_state=42)

In [45]:
rf_syn.fit(X_syn_train, y_syn_train)

RandomForestRegressor(n_estimators=50, random_state=42)

- 합성데이터 예측값

In [46]:
y_syn_pred = rf_syn.predict(X_syn_test)

In [47]:
index = X_syn_test.index.tolist()

In [48]:
df = pd.DataFrame({'index' : index ,'pred': y_syn_pred})

- 합성데이터 우범 회귀 예측값 상위 5%의 우범률

In [49]:
syn_5 = reg_syn_top_5per(df)

- 합성데이터 우범 회귀 예측값 상위 10%의 우범률

In [50]:
syn_10 = reg_syn_top_10per(df)

#### 우범률 정리 - 랜덤포레스트 

In [51]:
df_result_rf = pd.DataFrame({'category': 'RandomForestRegressor','base_5' : base_5, 'base_10': base_10,'syn_5': syn_5,'syn_10': syn_10})

선형 회귀분석을 활용할 때보다 지표의 격차비율이 감소했습니다. 

랜덤포레스트에서도 합성데이터가 잘 작동하는 것을 확인할 수 있었습니다. 

In [52]:
df_result_rf

,category,base_5,base_10,syn_5,syn_10
crime_yn,RandomForestRegressor,0.934615,0.831256,0.866667,0.642857


### xgBoost

마지막 알고리즘은 xgBoost입니다. 

XGBoost는 기존 그래디언트 부스팅 알고리즘의 성능을 개선하도록 설계된 강력한 기계 학습 알고리즘입니다.


주요 특징은 다음과 같습니다.


- 빠르고 효율적인 모델 교육을 위한 병렬 처리 및 하드웨어 최적화
- 과적합 방지 및 일반화 개선을 위한 정규화 기술
- 누락된 데이터 및 기능 선택 처리를 위한 기본 제공 지원



In [53]:
import xgboost as xgb

#### 원본데이터 (X_train, X_test, y_train, y_test)

- 원본데이터 xgBoost 회귀모델 학습 

In [54]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [55]:
params = {
    "max_depth": 2,
    "eta": 0.1,
    "subsample": 0.5,
    "colsample_bytree": 0.5,
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
}
num_round = 100

In [56]:
xgb_model = xgb.train(params, dtrain, num_round)

- 원본데이터 예측값

In [57]:
y_pred = xgb_model.predict(dtest)

In [58]:
index = X_test['dec_num'].index.tolist()

In [59]:
df = pd.DataFrame({'index' : index ,'pred': y_pred})

- 원본데이터 우범 회귀 예측값 상위 5%의 우범률

In [60]:
base_5 = reg_top_5per(df)

- 원본데이터 우범 회귀 예측값 상위 10%의 우범률

In [61]:
base_10 = reg_top_10per(df)

#### 합성데이터 (X_syn_train, X_syn_test, y_syn_train, y_syn_test)

- 합성데이터 xgBoost 회귀모델 학습 

다른 모델들과 다르게 xgb에서 갖고 있는 DMatrix 형태로 자료를 받아서 훈련을 진행합니다. 

In [62]:
dtrain_syn = xgb.DMatrix(X_syn_train, label=y_syn_train)
dtest_syn = xgb.DMatrix(X_syn_test, label=y_syn_test)

모델을 학습할때의 파라미터들 입니다. 

파라미터 조정을 통해서도 적합한 훈련 방법을 찾을 수 있습니다.

In [63]:
params = {
    "max_depth": 2,
    "eta": 0.1,
    "subsample": 0.5,
    "colsample_bytree": 0.5,
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
}
num_round = 100

파라미터와 데이터와 실행 횟수를 입력하여 학습을 실시합니다.

In [64]:
xgb_model_syn = xgb.train(params, dtrain_syn, num_round)

- 합성데이터 예측값

In [65]:
y_syn_pred = xgb_model_syn.predict(dtest_syn)

In [66]:
index = X_syn_test['dec_num'].index.tolist()

In [67]:
df = pd.DataFrame({'index' : index ,'pred': y_syn_pred})

- 합성데이터 우범 회귀 예측값 상위 5%의 우범률

In [68]:
syn_5 = reg_syn_top_5per(df)

- 합성데이터 우범 회귀 예측값 상위 10%의 우범률

In [69]:
syn_10 = reg_syn_top_10per(df)

#### 우범률 정리 - xgBoost 

In [70]:
df_result_xgb = pd.DataFrame({'category': 'xgboost','base_5' : base_5, 'base_10': base_10,'syn_5': syn_5,'syn_10': syn_10})

다른 알고리즘들과 비슷한 비율로 차이를 보이고 있습니다. 

In [71]:
df_result_xgb

,category,base_5,base_10,syn_5,syn_10
crime_yn,xgboost,0.409259,0.380556,0.6,0.466667


## 모델별 우범률 정리

모델별 우범률을 정리하였습니다. 

모델에 따라 분류기의 성능이 차이가 난다는 점과 5% 그룹에서 발생한 우범률과 10% 그룹에서 발생한 우범률을 비교할때

같이 감소하는 경향을 두 데이터에서 모두 확인할 수 있었습니다. 

이것은 같은 알고리즘을 활용해 학습을 진행할 경우 유사한 경향을 가진 결과를 얻을 수 있다는 것을 의미합니다.

제가 제시한 수치는 하나의 사례입니다.
CTGAN의 추출된 데이터의양, 생성되는 데이터의 양 등 다양한 파라미터에 의해서 통계적 특성이 변할 수 있습니다. 

In [72]:
total_result = pd.concat([df_result_lr, df_result_rf, df_result_xgb])

In [73]:
total_result = total_result.reset_index(drop=True)

In [74]:
total_result

,category,base_5,base_10,syn_5,syn_10
0,LinearRegression,0.350000,0.308333,0.266667,0.233333
1,RandomForestRegressor,0.934615,0.831256,0.866667,0.642857
2,xgboost,0.409259,0.380556,0.600000,0.466667
